In [1]:
from scipy.ndimage import imread
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from math import pi as π
import os
from scipy import ndimage
from numpy.linalg import det, inv
from svmutil import *

In [16]:
img = imread("negatives/n11.png")

In [4]:
def read_images(path):
    return [imread(path + name) for name in sorted(os.listdir(path)) if name.endswith(".png")]

def show_images(images):
    def subplot(grid, image, index, cmap=None):
        subplot = plt.subplot(grid[i*4+index])
        subplot.imshow(img, cmap=cmap)
        subplot.axis('off')
    
    grid = gridspec.GridSpec(len(images), 4, hspace=0., wspace=0., left=0, top=1, bottom=0, right=1)
    plt.figure(figsize=(100, 100))
    for i in range(len(images)):
        subplot(grid, images[i], 0)
        subplot(grid, images[i][:,:,0], 1, 'gray')
        subplot(grid, images[i][:,:,1], 2, 'gray')
        subplot(grid, images[i][:,:,2], 3, 'gray')
    grid.update(hspace=0.)
    plt.show()

def greyscale(image):
    return np.apply_along_axis(lambda x: sum(x) / 3, 2, image)
    
def rgb_features(images):
    return np.array([np.average(np.average(image, axis=0), axis=0) for image in images])

def histogram_feature(images):
    return np.array([ndimage.histogram(image, 0, 255, 5) for image in images])

def deviation_feature(images):
    return np.array([[ndimage.standard_deviation(image)] for image in images])

def laplace_feature(images):
    return np.array([[ndimage.minimum(ndimage.sobel(greyscale(image), mode='reflect'))] for image in images])

def features(images):
    f = np.concatenate((rgb_features(images), deviation_feature(images), laplace_feature(images)), axis=1)
    return f

In [8]:
negatives = np.array(read_images("../03/negatives/"))
positives = np.array(read_images("../03/positives/"))

In [230]:

num_samples = (len(features_0) + len(features_1))

errors = 0
confidence_sum = 0
for f in features_0[15:]:
    confidence_sum += confidence(f, µ_0, µ_1, Σ, φ)
    if is_parasite(f, µ_0, µ_1, Σ, φ):
        errors += 1
for f in features_1[15:]:
    confidence_sum += confidence(f, µ_0, µ_1, Σ, φ)
    if not is_parasite(f, µ_0, µ_1, Σ, φ):
        errors += 1

print("Model")
print("µ₀ \t", µ_0)
print("µ₁ \t", µ_1)
print("Σ \t", Σ)
print("φ \t", φ)

print("Error:", errors / num_samples)
print("Confidence:", confidence_sum / num_samples)

Model
µ₀ 	 [ 165.75069444  128.09398148  127.90520833   20.44602683  -41.93333333]
µ₁ 	 [ 179.88587963  144.90381944  147.94641204   24.30642702 -197.57777778]
Σ 	 [[   8116.59812886    9060.32057432    8200.28847656     994.64075342
   -35886.71608796]
 [   9060.32057432   11028.62910096    9625.21327984     734.50948492
   -40419.29996142]
 [   8200.28847656    9625.21327984    9420.98863812    1138.78109748
   -48429.72685185]
 [    994.64075342     734.50948492    1138.78109748     507.91402611
   -10200.7379977 ]
 [ -35886.71608796  -40419.29996142  -48429.72685185  -10200.7379977
   387142.93333333]]
φ 	 0.5
Error: 0.0
Confidence: [[  7.25912648e-13]]


In [17]:
y, x = svm_read_problem('heart_scale')
m = svm_train(y[:200], x[:200], '-c 4')
p_label, p_acc, p_val = svm_predict(y[200:], x[200:], m)

Accuracy = 84.2857% (59/70) (classification)


In [44]:
x = [list(l) for l in np.concatenate((features(positives), features(negatives)), axis=0)]
x_train = x[::2]
x_test = x[1::2]
y = [1] * len(positives) + [-1] * len(negatives)
y_train = y[::2]
y_test = y[1::2]


prob = svm_problem(y_train, x_train)
for t, t_name in enumerate(("linear", "polynomial", "radial-basis", "sigmoid")):
    param = svm_parameter('-t %d -c 4 -b 1' % t)
    m = svm_train(prob, param)
    print(t_name)
    p_label, p_acc, p_val = svm_predict(y_test, x_test, m)
    print()

linear
Model supports probability estimates, but disabled in predicton.
Accuracy = 100% (30/30) (classification)

polynomial
Model supports probability estimates, but disabled in predicton.
Accuracy = 100% (30/30) (classification)

radial-basis
Model supports probability estimates, but disabled in predicton.
Accuracy = 50% (15/30) (classification)

sigmoid
Model supports probability estimates, but disabled in predicton.
Accuracy = 50% (15/30) (classification)



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
